## This is a Comaprison and Testing of both simulators to The-State-Of-The-Art Simulator from Pennylane 

Both Simulators support the X, Z, I, H and CNOT gates, which makes them universal, they also support: `.draw()` which prints the sequence of gates used and on which quibts, `.Expval()` which computes the Expectation value of the whole circut on the Z-bases,`.measure()` which measures all qubits in the computational bases, and `.reset()` which resets the circuits.

the Naive Simulator supports saving scaled up tailored gates like CNOT from control x to target y in the memory which reduces computing time,this features can be turned on or off using the Parameter/Arg `keep_already_made_gates` , the default is `False` , after some testing , i found out that its better to keep it as `False` :) 

In [1]:
from Naive_Quantum_Circuits_Simulator import Naive_Circuit
from Tensor_Based_Quantum_Circuit_Sim import Tensor_Circuit 

import pennylane as qml 
import numpy as np 
import time 

lets try to construct the **bell state**: 

In [2]:
n_qubits = 2

Nc = Naive_Circuit(num_of_qubits=n_qubits) 
Tc = Tensor_Circuit(num_of_qubits=n_qubits)

Nc.H(0)
Nc.CNOT(0,1) 

Tc.H(0)
Tc.CNOT(0,1)


@qml.qnode(qml.device("lightning.qubit", wires=n_qubits))
def PennyL_circuit(return_type = 'probs'): 
    qml.Hadamard(0)
    qml.CNOT([0,1]) 
    if return_type == 'probs' :     
        return qml.probs(wires=[wire for wire in range(n_qubits)])
    
    elif return_type == 'Expval': 
        return [ qml.expval(qml.PauliZ(i)) for i in range(n_qubits) ] 
    
    else : 
        raise Exception("enter a valid return type please!")

In [3]:
PennyL_circuit()

tensor([0.5, 0. , 0. , 0.5], requires_grad=True)

We Can return Expectation values on the Z computationla bases too : 

In [4]:
print(PennyL_circuit(return_type="Expval")) 
print(Nc.ExpVal()) 
print(Tc.ExpVal())

[tensor(0., requires_grad=True), tensor(0., requires_grad=True)]
0.9999999999999998
0.9999999999999998


In [5]:
for sample in range(10) : 
    print("Naive Simulator    ","Tensor based Simulator")
    print(Nc.Measure() ," \t \t    ", Tc.Measure() ) 

Naive Simulator     Tensor based Simulator
00  	 	     11
Naive Simulator     Tensor based Simulator
11  	 	     11
Naive Simulator     Tensor based Simulator
00  	 	     11
Naive Simulator     Tensor based Simulator
00  	 	     11
Naive Simulator     Tensor based Simulator
00  	 	     11
Naive Simulator     Tensor based Simulator
11  	 	     11
Naive Simulator     Tensor based Simulator
00  	 	     11
Naive Simulator     Tensor based Simulator
11  	 	     00
Naive Simulator     Tensor based Simulator
11  	 	     00
Naive Simulator     Tensor based Simulator
00  	 	     00


Now lets test our tensor based simulator on 100 randomly placed CNOT gates : 

In [6]:
start_time = time.time()
big_tc = Tensor_Circuit(num_of_qubits = 25)

for i in range(100) :  # 100 CNOT gates test 
    try : 
        big_tc.CNOT( np.random.randint(25) , np.random.randint(25))
    except : 
        continue
    
print(time.time() - start_time)    

11.047212600708008


I tried to test the Naive Simulator , but its extremely inefficent that i could simulate more than 14 qubits : 
(a smalle note: i restart the kernal to free up some memory before any simulation)

In [6]:
start_time = time.time()
num_of_qubits = 14
big_nc = Naive_Circuit(num_of_qubits, keep_already_made_gates= False)

for i in range(100) :  # 100 CNOT gates test 
    try : 
        big_nc.CNOT( np.random.randint(num_of_qubits) , np.random.randint(num_of_qubits))
    except : 
        continue
    
print(time.time() - start_time)    

497.253555059433


### System Info: 

In [9]:
import platform
import cpuinfo
print(f"System: {platform.system()}") 
print(f"Machine: {platform.machine()}")  
cpu_info = cpuinfo.get_cpu_info()
print(f"CPU Brand: {cpu_info['brand_raw']}")       

System: Linux
Machine: x86_64
CPU Brand: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz
